In [1]:
from pytorch_libraries.model_NeuralStyleTransfer import *
from IPython.display import clear_output
from pylab import *
%matplotlib inline

In [10]:
# Image preprocessing
# For normalization, see https://github.com/pytorch/vision#models
transform = transforms.Compose([transforms.Resize(300),
                                transforms.ToTensor()])


content = Variable(load_image('../../data/images/chicago.jpg', transform ))
style   = Variable(load_image('../../data/images/wave.jpg', transform, shape = [content.size(3), content.size(2)] ))

target  = Variable(content.data.clone(), requires_grad=True)

vgg     = VGGNet().cuda()

In [11]:
optimizer = torch.optim.Adam([target], lr=1e-2)
STYLE_WEIGHT  = 1000.0
TOTAL_OPTSTEP = 100

before = target.clone().cpu().data.clamp_(0, 1)

In [12]:
for step in range(TOTAL_OPTSTEP):
        
        # Extract multiple(5) conv feature vectors
        target_features  = vgg(target)
        content_features = vgg(content)
        style_features   = vgg(style)

        style_loss = 0
        content_loss = 0
        
        # for each layer
        for f1, f2, f3 in zip(target_features, content_features, style_features):
            
            # Compute content loss (target and content image)
            content_loss += torch.mean((f1 - f2)**2) 
            # Reshape conv features
            _, c, h, w = f1.size()
            f1 = f1.view(c, h * w)
            f3 = f3.view(c, h * w)
            # Compute gram matrix  
            f1 = torch.mm(f1, f1.t())
            f3 = torch.mm(f3, f3.t())
            # Compute style loss (target and style image)
            style_loss += torch.mean((f1 - f3)**2) / (c * h * w) * STYLE_WEIGHT
            
        # Compute total loss, backprop and optimize
        loss = content_loss + style_loss 
        #loss =  style_loss 
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
                 
        if (step+1) % 10 == 0:
            after = target.clone().cpu().data.clamp_(0, 1)
            print ('Step [%d/%d], Content Loss: %.4f, Style Loss: %.4f, change %.4f' 
                   %(step+1, TOTAL_OPTSTEP, content_loss.data[0], style_loss.data[0], torch.mean(torch.abs(after-before))))
            before = after
    
        if (step+1) % 10 == 0:
            # Save the generated image
            #denorm = transforms.Normalize((-2.12, -2.04, -1.80), (4.37, 4.46, 4.44))
            img = target.clone().cpu().squeeze()
            img = img.data.clamp_(0, 1)
            torchvision.utils.save_image(img, '../../plot/nst/output-%d.png' %(step+1))
        

Step [10/100], Content Loss: 8.9336, Style Loss: 27267.9531, change 0.0477
Step [20/100], Content Loss: 11.7889, Style Loss: 9631.9404, change 0.0317
Step [30/100], Content Loss: 12.6432, Style Loss: 6260.2690, change 0.0175
Step [40/100], Content Loss: 12.9782, Style Loss: 4313.8877, change 0.0119
Step [50/100], Content Loss: 13.2979, Style Loss: 3273.8140, change 0.0088
Step [60/100], Content Loss: 13.5474, Style Loss: 2660.8677, change 0.0068
Step [70/100], Content Loss: 13.7177, Style Loss: 2262.3555, change 0.0055
Step [80/100], Content Loss: 13.8515, Style Loss: 1982.7922, change 0.0047
Step [90/100], Content Loss: 13.9541, Style Loss: 1773.3322, change 0.0041
Step [100/100], Content Loss: 14.0440, Style Loss: 1608.9413, change 0.0037


In [27]:
 models.vgg19(pretrained=True).features

Sequential (
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU (inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU (inplace)
  (4): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU (inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU (inplace)
  (9): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU (inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU (inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU (inplace)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU (inplace)
  (18): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (19): Conv2d(256,